In [2]:
# Load the autoreload extension
%load_ext autoreload
# Set autoreload to reload  imports only
# %autoreload 1
!pip install -U sentence-transformers
!pip install chromadb
!gdown 1-6DFFhW17XHcw2m1zz2v_0Su9FVjDUgW

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("/content/medium-articles.csv").sample(10000,random_state = 2023).reset_index(drop = True)

In [5]:
df.head()

,article_id,title,text,url,authors,timestamp,tags
0,42355,Women Reading : Notes on Henri Matisse,"Henri Matisse, Paris, 1913. Photo by Alvin Cob...",https://dooleyyoung.medium.com/women-reading-n...,['Kerry Dooley Young'],2021-01-01 03:28:05.017000+00:00,"['Matisse', 'History', 'Art', 'France', 'Paint..."
1,165859,"Greta Thunberg, a Rich White Girl from One of ...","Greta Thunberg, a Rich White Girl from One of ...",https://medium.com/@politicspeach/greta-thunbe...,['Charliepeach'],2019-09-24 23:19:33.241000+00:00,"['Democrats', 'Republicans', 'Greta Thunberg',..."
2,22517,Data Collaboratives: advancing open access for...,Brigid van Wanrooy (Swinburne University of Te...,https://medium.com/data-policy/data-collaborat...,['D P Blog Admin'],2020-04-15 16:54:59.878000+00:00,"['Open Data', 'Open Access', 'Policy', 'Data S..."
3,85500,"Apps, Tools and Resources to Improve Your Life",1. Wealthsimple\n\nWealthsimple offers a range...,https://medium.com/@joshevonyasyszczuk/apps-to...,"['Josh', 'Yasyszcjosh']",2020-12-18 02:31:00.265000+00:00,"['Lifestyle', 'Language Learning', 'University..."
4,52319,CVE-2020–25952,A Tale of SQL Injection Leads to admin panel b...,https://medium.com/bugbountywriteup/cve-2020-2...,['Mayur Parmar'],2020-11-17 02:05:06.518000+00:00,"['Cyber Defecers', 'Tryhackme', 'Cve 2020 2595..."


In [6]:
df.dtypes

article_id     int64
title         object
text          object
url           object
authors       object
timestamp     object
tags          object
dtype: object

# Data Preprocessing and Cleaning

In [7]:
df.authors = df.authors.apply(lambda x:"".join(eval(x)))

In [8]:
df.tags = df.tags.apply(lambda x:"".join(eval(x)))

In [9]:
df.sample(2)

,article_id,title,text,url,authors,timestamp,tags
6336,109459,The future of work is happening now thanks to ...,"Businesses, schools, and governments have all ...",https://medium.com/@danagardner/the-future-of-...,Dana Gardner,2020-12-22 16:08:26.987000+00:00,Remote WorkingInformation TechnologySecurityWo...
9710,152824,The Best of Doha’s Restaurants Delivering The ...,"By Doha News Staff\n\nFamily, friends and non-...",https://medium.com/dohanews/the-best-of-dohas-...,Editorial Team,2020-05-20 19:19:18.425000+00:00,RestaurantEid MubarakFoodQatarFoodies


## Model

In [10]:
model_name = "multi-qa-distilbert-cos-v1"

In [12]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer(model_name_or_path=model_name,device = 'cuda')

In [14]:
df.title.apply(lambda x : len(x.split()) ).describe()

count    10000.000000
mean         7.759000
std          3.888037
min          1.000000
25%          5.000000
50%          7.000000
75%         10.000000
max         42.000000
Name: title, dtype: float64

In [16]:
df.text.apply(lambda x : len(x.split()) ).describe()

count    10000.000000
mean       845.512500
std        836.938672
min          4.000000
25%        331.000000
50%        642.500000
75%       1109.000000
max      17005.000000
Name: text, dtype: float64

In [17]:
df.tags.apply(lambda x : len(x.split()) ).describe()

count    10000.000000
mean         2.832400
std          1.669548
min          1.000000
25%          2.000000
50%          3.000000
75%          4.000000
max         13.000000
Name: tags, dtype: float64

In [18]:
feature = df['title'] + df.text.apply(lambda x:"".join(x.split()[:400]))+ df.authors+df.tags

In [19]:
feature.head()

0    Women Reading : Notes on Henri MatisseHenriMat...
1    Greta Thunberg, a Rich White Girl from One of ...
2    Data Collaboratives: advancing open access for...
3    Apps, Tools and Resources to Improve Your Life...
4    CVE-2020–25952ATaleofSQLInjectionLeadstoadminp...
dtype: object

In [21]:
feature.iloc[0]

'Women Reading : Notes on Henri MatisseHenriMatisse,Paris,1913.PhotobyAlvinCoburn.Wikipediacopyofworkinpublicdomain.In2020,theMatissepaintingsthatcallmosttomearethosethatshowwomenreading.Bookshavebeenagreatsolaceforusthisyear.HowwonderfultobeabletoescapefromthenewsabouttheneedlessworseningofthepandemicandtheheatedclaimsfromtheWhiteHouse.Letmeknowinacommentwhichbooksprovidedyouarefugeofsomehoursfromthewoesof2020.Matisse,“ReadingWomanwithParasol,”1921.Wikiartcopyofimageinpublicdomain.Matisse,“WomanReading,”1921.Wikiartcopyofimageinpublicdomain.ResistanceFighterMyfavoriteMatissepaintingofareaderistheoneshownbelowofhisdaughter.Marguerite(1894–1982)ledquiteanadmirablelife.SheaidedtheFrenchResistanceduringWorldWarII.TheGestapoimprisonedherforthis1944.SheescapedduringtransporttoadeathcampandreturnedtoherfatherinsouthernFrance.Shetookcareofherfatherinhislateryearsandworkedtopreservehislegacyafterhisdeathin1954.Matisse,“PortraitdeMarguerite(TheReader,Margueritelisant,LaLiseuse),”1906.MuséedeGre

In [22]:
vectors = model.encode(feature,show_progress_bar = True,batch_size = 50)

Batches:   0%|          | 0/200 [00:00<?, ?it/s]

## Vector Database: Chroma DB

In [26]:
embedding = vectors.tolist()

In [27]:
import chromadb

In [28]:
client = chromadb.Client()
collection = client.create_collection('medium_article-metadata_db')

In [29]:
ids = df.article_id.astype(str).tolist()

In [30]:
metadata = df.to_dict(orient = 'records')  # List of Dictionaries


## Add Documents to the Vector Database

In [33]:
collection.add(ids=ids,embeddings = embedding, metadatas = metadata)

## Inference Pipeline

In [34]:
 current_read = df.sample(1,random_state = 10)
 current_read

,article_id,title,text,url,authors,timestamp,tags
937,158847,Great Way to Pamper Your Baby!,There are several important products that new ...,https://medium.com/@repanther/great-way-to-pam...,,2019-04-05 09:20:40.257000+00:00,BabyBaby SafetyParentingBaby Safety HarnessSafety


In [38]:
fe_req_id = current_read['article_id'].values[0]
fe_req_id

158847

In [39]:
current_read_metadata = collection.get(ids = [str(fe_req_id)],include=['embeddings'])

In [41]:
query = current_read_metadata['embeddings']

## Semantic Search

In [47]:
result = collection.query(query_embeddings=query,n_results = 5, where = {'article_id':{'$ne':int(fe_req_id)}})

In [48]:
result

{'ids': [['158835', '136093', '169667', '154475', '78755']],
 'distances': [[0.6112271547317505,
   0.6448383331298828,
   0.6875029802322388,
   0.6920519471168518,
   0.7258872985839844]],
 'metadatas': [[{'article_id': 158835,
    'authors': '',
    'tags': 'BabyDecorMust HaveNurseryBaby Nursery',
    'text': "Baby Nursery Essentials\n\nPlanning a nursery is a lot of work, and while most moms-and-dads-to-be get caught up in the decor details, it’s important to double-check that you’ve got the basics. From cribs to changing tables to curtains and baby monitors, we’ve rounded up a list of all the must-have baby essentials you’ll want to have ready for your little one’s nursery. They’re sure to make those upcoming days (and nights) a whole lot easier.\n\n1. Crib\n\nBaby crib and crib mattress or bedding\n\nOne of the most important baby nursery essentials is the baby’s bed. Whether you choose an elaborate crib or a simple bassinet, your little one’s sleeping spot will likely be the foc

In [49]:
recommendations = pd.DataFrame(result['metadatas'][0])

In [50]:
recommendations

,article_id,authors,tags,text,timestamp,title,url
0,158835,,BabyDecorMust HaveNurseryBaby Nursery,Baby Nursery Essentials\n\nPlanning a nursery ...,2020-12-02 03:31:44.771000+00:00,10 Baby Nursery Must-Haves: Your Essential Che...,https://medium.com/@mamapearofficial/10-baby-n...
1,136093,,FamilyFactsBabyParenting,We are sure that you already know a lot about ...,2019-04-11 20:49:51.953000+00:00,12 Truly Surprising Facts About Your Baby,https://medium.com/@rauchenwald_66280/12-truly...
2,169667,Fool The Bosses,Con ManConfessionsBabyPregnancySorry,"It’s not you, it’s definitely me. But it’s tru...",2020-10-15 05:04:37.014000+00:00,"Sorry, I can’t care that you’re having a baby.",https://medium.com/@foolthebosses/sorry-i-cant...
3,154475,,Baby StrollerBaby PramsBabyBaby Products,"best baby prams reviews,best baby stroller rev...",2020-05-25 18:15:29.781000+00:00,"best baby prams reviews,best baby stroller rev...",https://medium.com/@martinsharma2/best-baby-pr...
4,78755,Review Doctor,SleepBabyToddlersSleep DisordersBabysleep,Finally Proven !!!\n\nFind the Demonstrated Ar...,2020-12-20 04:13:10.754000+00:00,Baby sleep secrets !!!,https://medium.com/@reviewdoctor/baby-sleep-se...


In [52]:
recommendations["distance"] = result['distances'][0]

In [53]:
recommendations

,article_id,authors,tags,text,timestamp,title,url,distance
0,158835,,BabyDecorMust HaveNurseryBaby Nursery,Baby Nursery Essentials\n\nPlanning a nursery ...,2020-12-02 03:31:44.771000+00:00,10 Baby Nursery Must-Haves: Your Essential Che...,https://medium.com/@mamapearofficial/10-baby-n...,0.611227
1,136093,,FamilyFactsBabyParenting,We are sure that you already know a lot about ...,2019-04-11 20:49:51.953000+00:00,12 Truly Surprising Facts About Your Baby,https://medium.com/@rauchenwald_66280/12-truly...,0.644838
2,169667,Fool The Bosses,Con ManConfessionsBabyPregnancySorry,"It’s not you, it’s definitely me. But it’s tru...",2020-10-15 05:04:37.014000+00:00,"Sorry, I can’t care that you’re having a baby.",https://medium.com/@foolthebosses/sorry-i-cant...,0.687503
3,154475,,Baby StrollerBaby PramsBabyBaby Products,"best baby prams reviews,best baby stroller rev...",2020-05-25 18:15:29.781000+00:00,"best baby prams reviews,best baby stroller rev...",https://medium.com/@martinsharma2/best-baby-pr...,0.692052
4,78755,Review Doctor,SleepBabyToddlersSleep DisordersBabysleep,Finally Proven !!!\n\nFind the Demonstrated Ar...,2020-12-20 04:13:10.754000+00:00,Baby sleep secrets !!!,https://medium.com/@reviewdoctor/baby-sleep-se...,0.725887
